In [64]:
#Import Scraping Packages
import requests
from bs4 import BeautifulSoup
from time import sleep, time
from warnings import warn
from random import randint
from IPython.core.display import clear_output
import pandas as pd
import csv
import gc

C:\Users\jrtin\AppData\Local\Temp\ipykernel_14500\4019173959.py:7: DeprecationWarning: Importing clear_output from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import clear_output


In [116]:
def getPageHTML(url):
    user_agent = {'user-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers = user_agent)
    return response

In [148]:
def scrapeMetacriticGames(platform:str, pageStart:int, pageEnd:int, pageLimit:int = 80):

    #Prepare the monitoring of the loop
    pages = range(pageStart,pageEnd)
    start_time = time()

    #Prepare variables to create dictionary
    data_dict = []
    iGameCounter = 0

    #start loop
    for page in pages:
        url = f'https://www.metacritic.com/browse/games/release-date/available/{platform}/metascore?view=condensed&page={page}'
        response = getPageHTML(url)


        soup = BeautifulSoup(response.text,'html.parser')
        validity = soup.find('a', {"class": "title"})

        # Pause the loop
        sleep(randint(8,15))

        #Increment loop and check response validity
        elapsed_time = time() - start_time
        print(f'Request:{page}; Frequency: {page/elapsed_time} requests/s')
        clear_output(wait = True)
        
        #Break if no games are returned via request
        if validity is None:
            break
        else:
            for element in soup.find_all('div', {"class": "browse_list_wrapper"}):
                for item in element.find_all('tr', {"class": "expand_collapse"}):
                    data_dict.append({
                        'title': item.find('a', {"class": "title"}).find('h3').text,
                        'scores': {
                            'metaCritic': item.find('div', {"class": "metascore_w"}).text,
                            'userScore': item.find('div', {"class": "user"}).text
                        }           
                    })
                    iGameCounter += 1

        #Throw a warning for non-200 status codes
        if response.status_code != 200:
            warn(f'Request: {response}; Status code: {response.status_code}')
        
        #Break loop if number of requests is greater than expected
        if page > pageLimit:
            warn('Number of requests was greater than expected.')
            break
    return data_dict


In [149]:
data_dict = scrapeMetacriticGames('pc',0,10)


[{'title': 'Disco Elysium: The Final Cut', 'scores': {'metaCritic': '97', 'userScore': '8.2'}}, {'title': 'Half-Life 2', 'scores': {'metaCritic': '96', 'userScore': '9.2'}}, {'title': 'Grand Theft Auto V', 'scores': {'metaCritic': '96', 'userScore': '7.8'}}, {'title': 'Out of the Park Baseball 2007', 'scores': {'metaCritic': '96', 'userScore': '2.6'}}, {'title': 'The Orange Box', 'scores': {'metaCritic': '96', 'userScore': '9.1'}}, {'title': 'Half-Life', 'scores': {'metaCritic': '96', 'userScore': '9.1'}}, {'title': 'BioShock', 'scores': {'metaCritic': '96', 'userScore': '8.6'}}, {'title': "Baldur's Gate II: Shadows of Amn", 'scores': {'metaCritic': '95', 'userScore': '9.1'}}, {'title': 'Divinity: Original Sin II - Definitive Edition', 'scores': {'metaCritic': '95', 'userScore': '8.9'}}, {'title': 'Elden Ring', 'scores': {'metaCritic': '95', 'userScore': '6.7'}}, {'title': 'Portal 2', 'scores': {'metaCritic': '95', 'userScore': '9.1'}}, {'title': 'The Elder Scrolls V: Skyrim', 'scores'

In [153]:

df_gameData = pd.json_normalize(data_dict)
df_gameData.to_csv('metacriticData.csv', encoding='utf-8', sep='|')